In [7]:
from utilities import *
from sklearn import preprocessing
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.metrics import accuracy_score
from tqdm import tqdm

In [1]:
def initialisation(n0, n1, n2):
  ''' Initialisation des vecteurs Wi et bi'''
  W1 = np.random.randn(n1, n0) 
  b1 = np.random.randn(n1, 1)

  W2 = np.random.randn(n2, n1) 
  b2 = np.random.randn(n2, 1)

  parameters = {
    'W1' : W1,
    'b1' : b1,
    'W2' : W2,
    'b2' : b2
  }

  return parameters

In [2]:
def forward_propagation(X, parameters):
  '''Calcul de la matrice probabilités A'''
  W1 = parameters['W1']
  b1 = parameters['b1']
  W2 = parameters['W2']
  b2 = parameters['b2']
  
  Z1 = W1.dot(X) + b1
  A1 = 1 / (1 + np.exp(-Z1)) 
  
  Z2 = W2.dot(X) + b2
  A2 = 1 / (1 + np.exp(-Z2)) 
  
  activations = {
    'A1' : A1,
    'A2' : A2,
  }
  
  return activations

In [57]:
def log_loss(A, y):
  ''' Fonction coût, retourne un scalaire sur la véracité du modèle'''
  epsilon = 1e-15 # Pour ne jamais prendre log 0
  return 1/len(y) * np.sum(-y * np.log(A + epsilon) - (1 - y) * np.log(1-A + epsilon) )

In [3]:
def back_propagation(X, y, activations, parameters):
  ''' Calcul des gradients '''

  A1 = activations['A1']
  A2 = activations['A2']
  W2 = parameters['W2']

  m = y.shape[1]

  dZ2 = A2 - y
  dW2 = 1/ m * dZ2.dot(A1.T)
  db2 = 1/ m * np.sum(dZ2, axis=1, keepdims=True)

  dZ1 = np.dot(W2.T, dZ2) * A1 * (1 - A1)
  dW1 = 1/ m * dZ1.dot(X.T)
  db1 = 1/ m * np.sum(dZ1, axis=1, keepdims=True)

  gradients = {
    'dW1' : dW1,
    'db1' : db1,
    'dW2' : dW2,
    'db2' : db2
  }

  return gradients

In [4]:
def update(gradients, parameters, learning_rate):
  ''' Met à jour le modèle en ajustant les paramètres W et b à partir des gradients'''
  W1 = parameters['W1']
  b1 = parameters['b1']
  W2 = parameters['W2']
  b2 = parameters['b2']

  dW1 = gradients['dW1']
  db1 = gradients['db1']
  dW2 = gradients['dW2']
  db2 = gradients['db2']
  
  W1 = W1 - learning_rate * dW1
  b1 = b1 - learning_rate * db1
  W2 = W2 - learning_rate * dW2
  b2 = b2 - learning_rate * db2

  parameters = {
    'W1' : W1,
    'b1' : b1,
    'W2' : W2,
    'b2' : b2
  }

  return parameters

In [38]:
def predict(X, parameters):
  ''' Calcul le vecteur probas A pour un jeu de donnée X avec le modèle réglé aux paras W et b '''
  activations = forward_propagation(X, parameters)
  A2 = activations['A2']
  return A2 >= 0.5


In [5]:
def neural_network(X_train, y_train, X_test, y_test, n1, learning_rate=0.01, n_iter=1000):
  ''' Applique le procédé itératif : 
    -Calcul du vecteur colonne proba
    -Calcul l'erreur (le coût)
    -Calcul du vecteur colonne gradient W et du scalaire gradient b (càd l'évolution de l'erreur en fonction de W et b qui varient)
    -Mise à jour de W et b
  '''
  # Initialisation des para Wi et bi
  n0 = X_train.shape[0]
  n2 = y_train.shape[0]
  parametres = initialisation(n0, n1, n2)

  train_loss = []
  train_acc = []

  # Itération pour calculer W et b
  for i in tqdm(range(n_iter)):
    # Activation
    activations = forward_propagation(X_train, parametres)
    gradients = back_propagation(X_train, y_train, activations, parametres)
    update(gradients, parametres, learning_rate)

    if i % 10 == 0: # Limitation du nb de données à afficher (rapidité calcul)
      # Coût
      train_loss.append(log_loss(y_train, activations['A2'])) 
      y_pred = predict(X_train, parametres) # Calcul du vecteur colonne des prédictions
      current_accuracy = accuracy_score(y_train.flatten(), y_pred.flatten())
      train_acc.append(current_accuracy)
  
  plt.figure(figsize=(12, 4))
  
  plt.subplot(1, 2, 1)
  plt.plot(train_loss, label="Train loss")
  plt.legend()
  
  plt.subplot(1, 2, 2)
  plt.plot(train_acc, label="Train acc")
  plt.legend()

  plt.show()

  return parametres

